In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import RMSprop, Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
 
from skimage import io
from sklearn.model_selection import train_test_split

In [ ]:
image_list = os.listdir('imgs/')
print(len(image_list))

In [ ]:
def standarize_image(img_array,resize_pixels=256): 
  img = Image.fromarray(img_array)
 
  cols, rows = img.size
  extra = (rows-cols)/2
 
  if (extra>0):  
    crop_box = (0,extra,cols,cols+extra)
  else:
    crop_box = (-extra,0,rows-extra,rows) 
 
  standarized_image = img.crop(crop_box).resize((resize_pixels,resize_pixels), Image.ANTIALIAS)
 
  standarized_image_vector = (np.asarray(standarized_image).flatten().astype(np.float32)-128)/128 
 
  standarized_image_tensor = (np.asarray(standarized_image))
 
  return standarized_image_tensor, standarized_image_vector, standarized_image

In [ ]:
x_all = []

image_list = os.listdir('imgs/')
print(image_list)
    
for image in image_list:
    img_array = io.imread('imgs/' + image)
 
    (img_tensor,_,img) = standarize_image(img_array,128)
    x_all.append(img_tensor)

# Convert the list to a 4D array 
x_all = np.array(x_all)

In [ ]:
#(x_train, y_train, x_test, y_test) = train_test_split(x_all, y_all, , random_state=42)

#dataset = ((x_train, y_train), (x_test, y_test))
 
np.save('MRI_preprocessed',x_all)



In [ ]:
#((x_train, y_train), (x_test, y_test)) = np.load("/content/drive/My Drive/MRI_preprocessed.npy",allow_pickle=True)

x_train = np.load("MRI_preprocessed.npy",allow_pickle=True)

#X_train = np.concatenate((x_train, y_train))

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(x_train[101],cmap='gray')
print(x_train.shape)

In [ ]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)